In [22]:
%%file project1.py
import mrjob
from mrjob.job import MRJob
class MRCrimeSort(MRJob):
    def mapper(self, _, line):
        data_list = line.split(',')
        county=data_list[0][1:]
        state=data_list[1][-2:]
        crime_rate=data_list[2]
        yield(county,crime_rate)
        
if __name__ == '__main__':
    MRCrimeSort.run() 

Overwriting project1.py


In [23]:
!python project1.py -r local crime_rate.csv --output-dir=crime_output --no-output

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /var/folders/dd/15csrxdn2rscsl__0456fnrm0000gn/T/project1.yangyichieh.20211025.205138.108592
Running step 1 of 1...
job output is in crime_output
Removing temp directory /var/folders/dd/15csrxdn2rscsl__0456fnrm0000gn/T/project1.yangyichieh.20211025.205138.108592...


In [265]:
%%file crime_rate.py
import mrjob
from mrjob.job import MRJob
class crime_rate(MRJob):
    
    def mapper(self, _, line):
        data_list = line.split(',')
        county=data_list[0][1:]
        state=data_list[1][1:3]
        crime_rate=data_list[2]
        population = data_list[22]
        if population!="FIPS_ST":
            crime_pop = int(population)*float(crime_rate)/100000
            yield(state, (crime_pop,int(population)))
    def reducer(self,key,counts):
        count = list(counts)
        crime_pop=0
        population=0
        for c in count:
            crime_pop+=c[0]
            population+=c[1]
        result = crime_pop/population
        yield(key, result)

      
    
if __name__ == '__main__':
    crime_rate.run()

Overwriting crime_rate.py


In [266]:
!python crime_rate.py -r local crime_rate.csv --output-dir=crime_output --no-output

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /var/folders/dd/15csrxdn2rscsl__0456fnrm0000gn/T/crime_rate.yangyichieh.20211026.062232.847143
Running step 1 of 1...
job output is in crime_output
Removing temp directory /var/folders/dd/15csrxdn2rscsl__0456fnrm0000gn/T/crime_rate.yangyichieh.20211026.062232.847143...


In [13]:
%%file education.py
import mrjob
from mrjob.job import MRJob
class education(MRJob):
    
    def mapper(self, _, line):
        data_list=line.split("\t")
        state=data_list[1]
        ELevel_pop=data_list[32]
        ELevel=data_list[36]
        if ELevel!="" and ELevel!='"Percent of adults with less than a high school diploma, 2000"'\
            and ELevel_pop!="" and ELevel_pop!='"Less than a high school diploma, 2000"':
            ELevel=float(data_list[36])
            if len(ELevel_pop)>3:
                ELevel_pop=data_list[32][1:-1]
                ELevel_pop = ELevel_pop.replace(",","")
            else:
                ELevel_pop=data_list[32]
            ELevel_pop = int(ELevel_pop)
            #population = ELevel_pop*100/ELevel
            yield(state, (ELevel_pop,ELevel))

    def reducer(self, key, counts):
        count = list(counts)
        
        ELevel_pop=0        
        pop=0
        for c in count:
            pop += ((c[0]*100)/c[1])
            ELevel_pop+=c[0]
        yield(key,ELevel_pop/pop)
        
        
"""    
    def combiner(self, key, counts):
        count = list(counts)
        ELevel_pop=0
        ELevel=0
        for c in count:
            ELevel_pop+=c[0]
            population+=c[1]
        
        yield(key,(ELevel_pop,population))
    def reducer(self,key,counts):
        count = list(counts)
        ELevel_pop=0
        population=0
        for c in count:
            ELevel_pop+=c[0]
            population+=c[1]
        result =ELevel_pop/population
        yield(key, result)
    
        #if ELevel_pop!="Less than a high school diploma, 2015-19":
         #   all_pop = int(ELevel_pop)*100/float(ELevel)
            #yield(state, (all_pop,int(ELevel_pop)))
        #yield (ELevel, ELevel_pop)
    
"""       
    
if __name__ == '__main__':
    education.run()

Overwriting education.py


In [14]:
!python education.py -r local education.txt --output-dir=output --no-output

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /var/folders/dd/15csrxdn2rscsl__0456fnrm0000gn/T/education.yangyichieh.20211026.210342.281921
Running step 1 of 1...
job output is in output
Removing temp directory /var/folders/dd/15csrxdn2rscsl__0456fnrm0000gn/T/education.yangyichieh.20211026.210342.281921...
